# Movie Parsing

## Disclamer: this code takes a coulple of hours to run. 
## You can download parsed data [here](https://drive.google.com/open?id=1t0LNCbqLjiLkAMFwtP8OIYU-zPUCNAjK)


## OMDB
OMDB is Open Movie Database. Although, it is open, you will need to pay 1 doller to get the key and send up to 100k requests/day. For 5 you get access to the poster API.

http://www.omdbapi.com/

In [1]:
import pandas as pd
import requests
from tqdm import tqdm_notebook as tqdm
import json

myOmdbKey = 'your key here' # you need to buy omdb key for 1$ on patreon
movies = pd.read_csv('../../../../data/ml-20m/links.csv')
movies['imdbId'] = movies['imdbId'].apply(lambda i: '0' * (8 - len(str(i))) + str(i))
movies['tmdbId'] = movies['tmdbId'].fillna(-1).astype(int).apply(str)
movies = movies.set_index('movieId')
movies = movies.to_dict(orient='index')

> If failed pops up, run this block again till it's done

In [44]:
# movies = json.load(open("../../../../data/parsed/omdb.json", "r") )

In [9]:
for id in tqdm(movies.keys()):
    imdbId = movies[id]['imdbId']
    if movies[id].get('omdb', False):
        continue
    try:
        movies[id]['omdb'] = requests.get("http://www.omdbapi.com/?i=tt{}&apikey={}&plot=full".format(imdbId,
                                                                                         myOmdbKey)).json()
    except:
        print(id, imdbId, 'failed')

In [17]:
with open("../../../../data/parsed/omdb.json", "w") as write_file:
    json.dump(movies, write_file)

In [46]:
movies['1']

{'imdbId': '00114709',
 'tmdbId': '862',
 'omdb': {'Title': 'Toy Story',
  'Year': '1995',
  'Rated': 'G',
  'Released': '22 Nov 1995',
  'Runtime': '81 min',
  'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
  'Director': 'John Lasseter',
  'Writer': 'John Lasseter (original story by), Pete Docter (original story by), Andrew Stanton (original story by), Joe Ranft (original story by), Joss Whedon (screenplay by), Andrew Stanton (screenplay by), Joel Cohen (screenplay by), Alec Sokolow (screenplay by)',
  'Actors': 'Tom Hanks, Tim Allen, Don Rickles, Jim Varney',
  'Plot': 'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure

## TMDB

In [25]:
import pandas as pd
import requests
from tqdm import tqdm_notebook as tqdm
import json

myTmdbKey = 'your key here' # you can get it for free if you ask them nicely
movies = pd.read_csv('../../../../data/ml-20m/links.csv')
movies['imdbId'] = movies['imdbId'].apply(lambda i: '0' * (8 - len(str(i))) + str(i))
movies['tmdbId'] = movies['tmdbId'].fillna(-1).astype(int).apply(str)
movies = movies.set_index('movieId')
movies = movies.to_dict(orient='index')

In [26]:
import asyncio
# ! pip install aiohttp --user
import aiohttp
# ! pip install asyncio-throttle --user
from asyncio_throttle import Throttler

In [27]:
# movies = json.load(open("../../../../data/parsed/tmdb.json", "r") )

> you can also run this code multiple times

In [28]:
throttler = Throttler(rate_limit=4, period=2)

async def tmdb(session, id, tmdbId):
    url = "https://api.themoviedb.org/3/movie/{}?api_key={}".format(tmdbId, myTmdbKey)
    async with throttler:
        async with session.get(url) as resp:
            if resp.status == 429:
                print('throttling')
                await asyncio.sleep(0.2)
            
            movies[id]['tmdb'] = await resp.json()
    
    # this also controlls the timespan between calls
    await asyncio.sleep(0.05)
    

async def main():
    async with aiohttp.ClientSession() as session:
        for id in tqdm(movies.keys()):
            tmdbId = movies[id]['tmdbId']
            if movies[id].get('tmdb', False) and 'status_code' not in movies[id]['tmdb']:
                continue
            await tmdb(session, id, tmdbId)
        
        
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.create_task(main())

In [31]:
with open("../../../../data/parsed/tmdb.json", "w") as write_file:
    json.dump(movies, write_file)

In [43]:
movies['1']

{'imdbId': '00114709',
 'tmdbId': '862',
 'tmdb': {'adult': False,
  'backdrop_path': '/dji4Fm0gCDVb9DQQMRvAI8YNnTz.jpg',
  'belongs_to_collection': {'id': 10194,
   'name': 'Toy Story Collection',
   'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
   'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'},
  'budget': 30000000,
  'genres': [{'id': 16, 'name': 'Animation'},
   {'id': 35, 'name': 'Comedy'},
   {'id': 10751, 'name': 'Family'}],
  'homepage': 'http://toystory.disney.com/toy-story',
  'id': 862,
  'imdb_id': 'tt0114709',
  'original_language': 'en',
  'original_title': 'Toy Story',
  'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
  'popularity': 29.3,
  'poster_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
  